# Spark LLM Assistant

## Initialization

In [1]:
from langchain.chat_models import ChatOpenAI
from spark_llm import SparkLLMAssistant

llm = ChatOpenAI(model_name='gpt-4') # using gpt-4 can achieve better results
assistant=SparkLLMAssistant(llm=llm, verbose=True)
assistant.activate() # active partial functions for Spark DataFrame

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/21 12:02:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Example 1: Auto sales by brand in US 2022

In [2]:
# Search and ingest web content into a DataFrame
auto_df = assistant.create_df("2022 USA national auto sales by brand")
auto_df.show()

INFO: Parsing URL: https://www.carpro.com/blog/full-year-2022-national-auto-sales-by-brand

INFO: SQL query for the ingestion:

INFO: CREATE OR REPLACE TEMP VIEW auto_sales_2022 AS SELECT * FROM VALUES
(1, 'Toyota', 1849751, -9),
(2, 'Ford', 1767439, -2),
(3, 'Chevrolet', 1502389, 6),
(4, 'Honda', 881201, -33),
(5, 'Hyundai', 724265, -2),
(6, 'Kia', 693549, -1),
(7, 'Jeep', 684612, -12),
(8, 'Nissan', 682731, -25),
(9, 'Subaru', 556581, -5),
(10, 'Ram Trucks', 545194, -16),
(11, 'GMC', 517649, 7),
(12, 'Mercedes-Benz', 350949, 7),
(13, 'BMW', 332388, -1),
(14, 'Volkswagen', 301069, -20),
(15, 'Mazda', 294908, -11),
(16, 'Lexus', 258704, -15),
(17, 'Dodge', 190793, -12),
(18, 'Audi', 186875, -5),
(19, 'Cadillac', 134726, 14),
(20, 'Chrysler', 112713, -2),
(21, 'Buick', 103519, -42),
(22, 'Acura', 102306, -35),
(23, 'Volvo', 102038, -16),
(24, 'Mitsubishi', 102037, -16),
(25, 'Lincoln', 83486, -4),
(26, 'Porsche', 70065, 0),
(27, 'Genesis', 56410, 14),
(28, 'INFINITI', 46619, -20),
(29, 

+----+-------------+--------+-------+
|rank|        brand|us_sales|vs_2021|
+----+-------------+--------+-------+
|   1|       Toyota| 1849751|     -9|
|   2|         Ford| 1767439|     -2|
|   3|    Chevrolet| 1502389|      6|
|   4|        Honda|  881201|    -33|
|   5|      Hyundai|  724265|     -2|
|   6|          Kia|  693549|     -1|
|   7|         Jeep|  684612|    -12|
|   8|       Nissan|  682731|    -25|
|   9|       Subaru|  556581|     -5|
|  10|   Ram Trucks|  545194|    -16|
|  11|          GMC|  517649|      7|
|  12|Mercedes-Benz|  350949|      7|
|  13|          BMW|  332388|     -1|
|  14|   Volkswagen|  301069|    -20|
|  15|        Mazda|  294908|    -11|
|  16|        Lexus|  258704|    -15|
|  17|        Dodge|  190793|    -12|
|  18|         Audi|  186875|     -5|
|  19|     Cadillac|  134726|     14|
|  20|     Chrysler|  112713|     -2|
+----+-------------+--------+-------+
only showing top 20 rows



In [5]:
auto_df.llm.plot()

INFO: To visualize the result of `df` using Plotly, you can follow these steps:

1. First, make sure you have the Plotly package installed. You can install it using `!pip install plotly` if you are using Jupyter Notebook or `pip install plotly` in your terminal.

2. Convert the PySpark DataFrame to a Pandas DataFrame.

3. Create a plot using Plotly's graph objects.

Here's the code:


```
import plotly.graph_objects as go
import plotly.express as px

# Convert the PySpark DataFrame to a Pandas DataFrame
pdf = df.toPandas()

# Create a bar plot using Plotly
fig = px.bar(pdf, x='brand', y='us_sales', text='rank', title='US Auto Sales 2022')

# Customize the plot
fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.update_layout(xaxis_title='Brand', yaxis_title='US Sales')

# Show the plot
fig.show()
```

This code will create a bar plot with the brand on the x-axis and US sales on the y-axis. The rank of each brand will be displayed above each bar. The plot will be direc

In [6]:
auto_df.llm.plot("pie char for market shares, show the top 5 brands and the sum of others")

INFO: To visualize the result of the `df` using Plotly, you can follow these steps:

1. Convert the PySpark DataFrame to a Pandas DataFrame.
2. Filter the top 5 brands and calculate the sum of others.
3. Create a pie chart using Plotly and show the market shares.

Here's the code to achieve that:


```
import pandas as pd
import plotly.graph_objs as go

# Convert the PySpark DataFrame to a Pandas DataFrame
df_pd = df.toPandas()

# Filter the top 5 brands and calculate the sum of others
top_5_brands = df_pd.nlargest(5, 'us_sales')
other_brands_total = df_pd[~df_pd.index.isin(top_5_brands.index)]['us_sales'].sum()

# Create a pie chart using Plotly
labels = list(top_5_brands['brand']) + ['Others']
values = list(top_5_brands['us_sales']) + [other_brands_total]

fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3)])
fig.update_layout(title='Market Shares of Top 5 Auto Brands in 2022')

# Show the plot directly
fig.show()
```

This code will create a pie chart that shows the 

In [9]:
# Apply transforms to a Dataframe
auto_top_growth_df=auto_df.llm.transform("top brand with the highest growth")
auto_top_growth_df.show()

INFO: SQL query for the transform:
INFO: SELECT brand
FROM temp_view_for_transform
ORDER BY vs_2021 DESC
LIMIT 1



+-------+
|  brand|
+-------+
|Genesis|
+-------+



In [10]:
# Explain what a DataFrame is retrieving.
auto_top_growth_df.llm.explain()

'In summary, this dataframe is retrieving the brand with the highest value for the `vs_2021` metric from a temporary view named `temp_view_for_transform`. The result will only include the top brand based on the `vs_2021` value.'

In [11]:
auto_top_growth_df.llm.verify("expect sales change percentage to be between -100 to 100")

INFO: Generated code:
INFO: def has_sales_change_in_range(df) -> bool:
    from pyspark.sql.functions import col

    # Check if the dataframe has the column "sales_change_percentage"
    if "sales_change_percentage" not in df.columns:
        return False

    # Check if the sales change percentage is between -100 and 100
    df_filtered = df.filter((col("sales_change_percentage") >= -100) & (col("sales_change_percentage") <= 100))

    # If the number of rows in the filtered dataframe is equal to the number of rows in the original dataframe,
    # it means all the sales change percentages are in the valid range
    if df_filtered.count() == df.count():
        return True
    else:
        return False

result = has_sales_change_in_range(df)

INFO: 
Result: False


## Example 2: USA Presidents

In [12]:
# You can also specify the expected columns for the ingestion.
df=assistant.create_df("USA presidents", ["president", "vice_president"])
df.show()

INFO: Parsing URL: https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States

INFO: SQL query for the ingestion:

INFO: CREATE OR REPLACE TEMP VIEW presidents AS SELECT * FROM VALUES 
('George Washington', 'John Adams'),
('John Adams', 'Thomas Jefferson'),
('Thomas Jefferson', 'Aaron Burr'),
('Thomas Jefferson', 'George Clinton'),
('James Madison', 'George Clinton'),
('James Madison', 'Elbridge Gerry'),
('James Monroe', 'Daniel D. Tompkins'),
('John Quincy Adams', 'John C. Calhoun'),
('Andrew Jackson', 'John C. Calhoun'),
('Andrew Jackson', 'Martin Van Buren'),
('Martin Van Buren', 'Richard Mentor Johnson'),
('William Henry Harrison', 'John Tyler'),
('John Tyler', NULL),
('James K. Polk', 'George M. Dallas'),
('Zachary Taylor', 'Millard Fillmore'),
('Millard Fillmore', NULL),
('Franklin Pierce', 'William R. King'),
('James Buchanan', 'John C. Breckinridge'),
('Abraham Lincoln', 'Hannibal Hamlin'),
('Abraham Lincoln', 'Andrew Johnson'),
('Andrew Johnson', NULL),
('Ulysses S.

+--------------------+--------------------+
|           president|      vice_president|
+--------------------+--------------------+
|   George Washington|          John Adams|
|          John Adams|    Thomas Jefferson|
|    Thomas Jefferson|          Aaron Burr|
|    Thomas Jefferson|      George Clinton|
|       James Madison|      George Clinton|
|       James Madison|      Elbridge Gerry|
|        James Monroe|  Daniel D. Tompkins|
|   John Quincy Adams|     John C. Calhoun|
|      Andrew Jackson|     John C. Calhoun|
|      Andrew Jackson|    Martin Van Buren|
|    Martin Van Buren|Richard Mentor Jo...|
|William Henry Har...|          John Tyler|
|          John Tyler|                null|
|       James K. Polk|    George M. Dallas|
|      Zachary Taylor|    Millard Fillmore|
|    Millard Fillmore|                null|
|     Franklin Pierce|     William R. King|
|      James Buchanan|John C. Breckinridge|
|     Abraham Lincoln|     Hannibal Hamlin|
|     Abraham Lincoln|      Andr

In [13]:
presidents_who_were_vp = df.llm.transform("presidents who were also vice presidents")
presidents_who_were_vp.show()

INFO: SQL query for the transform:
INFO: SELECT DISTINCT president
FROM temp_view_for_transform
WHERE president IN (SELECT vice_president FROM temp_view_for_transform)



+------------------+
|         president|
+------------------+
|        John Adams|
|  Thomas Jefferson|
|  Martin Van Buren|
|  Millard Fillmore|
|        John Tyler|
|    Andrew Johnson|
| Chester A. Arthur|
|Theodore Roosevelt|
|   Calvin Coolidge|
|   Harry S. Truman|
+------------------+



In [14]:
presidents_who_were_vp.llm.explain()

'In summary, this dataframe is retrieving the distinct list of presidents who have never been vice presidents from the `presidents` table.'

In [15]:
presidents_who_were_vp.llm.verify("expect no NULL values")

INFO: Generated code:
INFO: def has_no_null_values(df) -> bool:
    from pyspark.sql.functions import col
    
    # Check if there are any NULL values in the DataFrame
    null_counts = df.select([col(c).isNull().alias(c) for c in df.columns]).filter(" or ".join(df.columns)).count()

    # If there are no NULL values, return True, otherwise return False
    if null_counts == 0:
        return True
    else:
        return False

result = has_no_null_values(df)

INFO: 
Result: True


# Example 3: Top 10 tech companies

In [16]:
# Search and ingest web content into a DataFrame
company_df=assistant.create_df("Top 10 tech companies by market cap", ['company', 'cap', 'country'])
company_df.show()

INFO: Parsing URL: https://www.statista.com/statistics/1350976/leading-tech-companies-worldwide-by-market-cap/

INFO: SQL query for the ingestion:

INFO: CREATE OR REPLACE TEMP VIEW top_tech_companies AS SELECT * FROM VALUES
('Apple', 2242, 'United States'),
('Microsoft', 1821, 'United States'),
('Alphabet (Google)', 1229, 'United States'),
('Amazon', 902.4, 'United States'),
('Tesla', 541.4, 'United States'),
('TSMC', 410.9, 'Taiwan'),
('NVIDIA', 401.7, 'United States'),
('Tencent', 377.8, 'China'),
('Meta Platforms (Facebook)', 302.1, 'United States'),
('Samsung', 301.7, 'South Korea')
AS top_tech_companies(company, cap, country)

INFO: Storing data into temp view: top_tech_companies



+--------------------+------+-------------+
|             company|   cap|      country|
+--------------------+------+-------------+
|               Apple|2242.0|United States|
|           Microsoft|1821.0|United States|
|   Alphabet (Google)|1229.0|United States|
|              Amazon| 902.4|United States|
|               Tesla| 541.4|United States|
|                TSMC| 410.9|       Taiwan|
|              NVIDIA| 401.7|United States|
|             Tencent| 377.8|        China|
|Meta Platforms (F...| 302.1|United States|
|             Samsung| 301.7|  South Korea|
+--------------------+------+-------------+



In [17]:
us_company_df=company_df.llm.transform("companies in United States")
us_company_df.show()

INFO: SQL query for the transform:
INFO: SELECT company, cap, country
FROM temp_view_for_transform
WHERE country = 'United States'



+--------------------+------+-------------+
|             company|   cap|      country|
+--------------------+------+-------------+
|               Apple|2242.0|United States|
|           Microsoft|1821.0|United States|
|   Alphabet (Google)|1229.0|United States|
|              Amazon| 902.4|United States|
|               Tesla| 541.4|United States|
|              NVIDIA| 401.7|United States|
|Meta Platforms (F...| 302.1|United States|
+--------------------+------+-------------+



In [18]:
us_company_df.llm.explain()

'In summary, this dataframe is retrieving the company name, market capitalization, and country for all the top tech companies that are based in the United States.'

In [19]:
us_company_df.llm.verify("expect all company names to be unique")

INFO: Generated code:
INFO: def has_unique_company_names(df) -> bool:
    from pyspark.sql.functions import countDistinct

    # Get the number of unique company names
    unique_companies = df.agg(countDistinct("company")).collect()[0][0]

    # Get the total number of rows in the DataFrame
    total_rows = df.count()

    # Check if the number of unique company names is equal to the total number of rows
    if unique_companies == total_rows:
        return True
    else:
        return False

result = has_unique_company_names(df)

INFO: 
Result: True


## Example 4: Ingestion from a URL
Instead of searching for the web page, you can also ask the assistant to ingest from a URL.

In [20]:
best_albums_df = assistant.create_df('https://time.com/6235186/best-albums-2022/', ["album", "artist", "year"])
best_albums_df.show()

INFO: Parsing URL: https://time.com/6235186/best-albums-2022/

INFO: SQL query for the ingestion:

INFO: CREATE OR REPLACE TEMP VIEW best_albums_2022 AS SELECT * FROM VALUES
('Motomami', 'Rosalía', 2022),
('You Can\'t Kill Me', '070 Shake', 2022),
('Mr. Morale & The Big Steppers', 'Kendrick Lamar', 2022),
('Big Time', 'Angel Olsen', 2022),
('Electricity', 'Ibibio Sound Machine', 2022),
('It\'s Almost Dry', 'Pusha T', 2022),
('Chloe and the Next 20th Century', 'Father John Misty', 2022),
('Renaissance', 'Beyoncé', 2022),
('19 Masters', 'Saya Gray', 2022),
('Un Verano Sin Ti', 'Bad Bunny', 2022)
AS v1(album, artist, year)

INFO: Storing data into temp view: best_albums_2022



+--------------------+--------------------+----+
|               album|              artist|year|
+--------------------+--------------------+----+
|            Motomami|             Rosalía|2022|
|   You Can't Kill Me|           070 Shake|2022|
|Mr. Morale & The ...|      Kendrick Lamar|2022|
|            Big Time|         Angel Olsen|2022|
|         Electricity|Ibibio Sound Machine|2022|
|     It's Almost Dry|             Pusha T|2022|
|Chloe and the Nex...|   Father John Misty|2022|
|         Renaissance|             Beyoncé|2022|
|          19 Masters|           Saya Gray|2022|
|    Un Verano Sin Ti|           Bad Bunny|2022|
+--------------------+--------------------+----+



In [21]:
best_albums_df.llm.verify("expect each year to be 2022")

INFO: Generated code:
INFO: def has_2022_years(df) -> bool:
    from pyspark.sql.functions import col

    # Filter the rows where the year is not 2022
    not_2022_rows = df.filter(col("year") != 2022)

    # Check if there are any rows with a year other than 2022
    if not_2022_rows.count() == 0:
        return True
    else:
        return False

result = has_2022_years(df)

INFO: 
Result: True


## Example 5: UDF Generation

You can also ask the assistant to generate code for a Spark UDF by providing.

In [22]:
@assistant.udf
def convert_grades(grade_percent: float) -> str:
    """Convert the grade percent to a letter grade using standard cutoffs"""
    ...

INFO: Creating following Python UDF:
INFO: def convert_grades(grade_percent: float) -> str:
    if grade_percent is not None:
        if grade_percent >= 90.0:
            return "A"
        elif grade_percent >= 80.0:
            return "B"
        elif grade_percent >= 70.0:
            return "C"
        elif grade_percent >= 60.0:
            return "D"
        else:
            return "F"



In [23]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
spark.udf.register("convert_grades", convert_grades)
percentGrades = [(1, 97.8), (2, 72.3), (3, 81.2)]
df = spark.createDataFrame(percentGrades, ["student_id", "grade_percent"])
df.selectExpr("student_id", "convert_grades(grade_percent)").show()

+----------+-----------------------------+
|student_id|convert_grades(grade_percent)|
+----------+-----------------------------+
|         1|                            A|
|         2|                            C|
|         3|                            B|
+----------+-----------------------------+



# Cache
The SparkLLMAssistant supports a simple in-memory and persistent cache system. It keeps an in-memory staging cache, which gets updated for LLM and web search results. The staging cache can be persisted through the commit() method. Cache lookup is always performed on the persistent cache only.

In [7]:
assistant.commit()